note:

- using one-hot with sklearn, xgboost requested

In [57]:
# import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import partial

import xgboost as xgb
import os
from time import time

get_ipython().run_line_magic('matplotlib', 'inline')

In [2]:
print(os.listdir("./input"))

['.DS_Store', 'rossmann-store-sales', '.ipynb_checkpoints']


## Data EDA

In [3]:
# read data

train = pd.read_csv("./input/rossmann-store-sales/train.csv", parse_dates=[2], low_memory=False)
test = pd.read_csv("./input/rossmann-store-sales/test.csv", parse_dates=[3], low_memory=False)
store = pd.read_csv("./input/rossmann-store-sales/store.csv", low_memory=False)

## Data Cleaning

In [4]:
# the store should be open in the test,so we fillna with 1
test.fillna(1, inplace=True)

# fillna in store with 0 has better result than median()
store.fillna(0, inplace=True)

In [5]:
display(train.isnull().sum(),test.isnull().sum(),store.isnull().sum())

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [6]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [7]:
# split the last 6 weeks data as hold-out set (idea from Gert https://www.kaggle.com/c/rossmann-store-sales/discussion/18024)
train = train.sort_values(['Date'],ascending = False)

x_hold = train[:6*7*1115]
x_train = train[6*7*1115:]

In [8]:
# only use data of Sales > 0 and Open is 1
# x_hold = x_hold[x_hold["Open"] != 0]
x_hold = x_hold[x_hold["Sales"] > 0]
# x_train = x_train[x_train["Open"] != 0]
x_train = x_train[x_train["Sales"] > 0]

In [51]:
# add one-hot code cate
cols_cate = ["StoreType", "Assortment", "StateHoliday"]

onehot_encoder = OneHotEncoder()
onehot_encoder.fit(x_train[cols_cate].append(x_hold[cols_cate]).append(test[cols_cate]))

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [65]:
def covert_seq(seq):
    """add promo seq"""
    new_seq = list()
    j = 0
    last = None
    for i in seq:
        if last == 0 and i == 1:
            j = 0
        if i == 0:
            new_seq.append(10)
        elif i == 1:
            j += 1
            new_seq.append(j)
        else:
            raise
        last = i
    return new_seq  

def features_create(data:pd.DataFrame, onehot_encoder:OneHotEncoder, cate_features:list):    
    data = data.copy()
    
    # add one-hot code cate
    data_oh_features = onehot_encoder.transform(data[cate_features]).toarray()
    df_oh_features = pd.DataFrame(data_oh_features, columns=onehot_encoder.get_feature_names(cate_features), index=data.index)
    
    del data_oh_features
    data = data.drop(cate_features, 1).join(df_oh_features)
    
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) +         (data.Month - data.CompetitionOpenSinceMonth)
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) +         (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)        
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    data["CompetitionDistance_log1p"] = data["CompetitionDistance"].apply(np.log1p)
    
    # add promo seq
    data_promo = data.set_index(["Store", "Date"])["Promo"].copy()
    data_promo.sort_index(level=[0, 1], ascending=[True, True], inplace=True)
    data_promo = data_promo.to_frame()
    data_promo = data_promo.groupby(data_promo.index.get_level_values(0)).apply(lambda x: x.assign(Promo_seq=covert_seq(x["Promo"]))).reset_index()
    data = data.merge(data_promo, on=["Store", "Date"], how="left")
    
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1
    return data

In [66]:
features_create = partial(features_create, onehot_encoder=onehot_encoder, cate_features=cols_cate)

In [67]:
x_train = features_create(x_train)
x_hold = features_create(x_hold)
x_test = features_create(test)
print('Features creation finished')

Features creation finished


In [14]:
# drop the features of not help
drop_features = ['Date','Customers','Open','PromoInterval','monthStr','CompetitionDistance']
x_train.drop(drop_features, axis=1, inplace=True)
x_hold.drop(drop_features, axis=1, inplace =True)

test_drop_features = set(drop_features) & set(x_test) | set(["Id"]) 
x_test.drop(test_drop_features, 1, inplace=True)

In [15]:
y_train = np.log1p(x_train.Sales)
x_train = x_train.drop(['Sales'], axis=1)
y_hold = np.log1p(x_hold.Sales)
x_hold = x_hold.drop(['Sales'], axis=1)

## modelling

In [16]:
# define rmspe for xgb(code from https://www.kaggle.com/cast42/xgboost-in-python-with-rmspe-v2/code)
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [18]:
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.015,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1301
          }

params['gpu_id'] = 0
params['tree_method'] = 'gpu_hist'

num_boost_round = 6000
early_stopping_rounds = 100

In [19]:

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_hold, y_hold)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

# training model
print("Train a XGBoost model")
start = time()
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, 
  early_stopping_rounds=early_stopping_rounds, feval=rmspe_xg, verbose_eval=100)
end = time()
print('Training time is {:2f} s.'.format(end-start))

# save model
gbm.save_model("0001.model")

# checking with hold-on dataset
print("validating")
x_hold.sort_index(inplace=True) 
y_hold.sort_index(inplace=True) 

yhat = gbm.predict(xgb.DMatrix(x_hold))
error = rmspe(np.expm1(y_hold), np.expm1(yhat))

print('RMSPE: {:.6f}'.format(error))

Train a XGBoost model
[0]	train-rmse:8.14457	eval-rmse:8.15156	train-rmspe:0.999851	eval-rmspe:0.999853
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.


In [ ]:
# analysis by hold-out set
res = pd.DataFrame(data = y_hold)
res['Prediction'] = yhat
res = pd.merge(x_hold, res, left_index= True, right_index=True)
res['Prediction_True'] = res['Prediction'].apply(np.expm1)
res['Sales_True'] = res['Sales'].apply(np.expm1)
res['Ratio'] = res.Prediction / res.Sales
res['Ratio_True'] = res.Prediction.apply(np.expm1) / res.Sales.apply(np.expm1)
res['Error'] = abs(res.Ratio-1)
res['Weight'] = res.Sales / res.Prediction

res.to_csv("res.csv", index=False)
res.head()

In [ ]:
# whole correction

def correction(y_true, y_predict):
    W = [(0.990 + (i/1000)) for i in range(20)]
    score = []
    for w in W:
        error = rmspe(np.expm1(y_true), np.expm1(y_predict * w))
        score.append(error)
    score = pd.Series(score, index=W)
    best_weight = score[score == score.min()].index
    return best_weight

def correction_by_store(x_hold, x_test):
    store_no = x_hold["Store"].unique()
    
    weight_y_test = np.zeros(len(x_test))
    weight_y_hold = np.zeros(len(x_hold))

    store_weights = {}
    
    for no in store_no:
        df = x_hold[x_hold["Store"] == no]
        y_pred = df.Prediction
        y_true = df.Sales
        
        best_weight = correction(y_true, y_pred)
        store_weights[no] = best_weight
        
        weight_y_test[x_test.Store == no] = best_weight
        weight_y_hold[x_hold.Store == no] = best_weight

    return weight_y_hold, weight_y_test

In [ ]:
w_hold, w_test = correction_by_store(res, x_test)

In [ ]:
yhat_new = yhat * w_hold
error = rmspe(np.expm1(y_hold), np.expm1(yhat_new))
print('RMSPE for weight corretion {:6f}'.format(error))

In [ ]:
print("Make predictions on the test set")
dtest = xgb.DMatrix(x_test)
test_probs = gbm.predict(dtest)

# model1  kaggle private score
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs)})
result.to_csv("Rossmann_submission_1.csv", index=False)

# model2 kaggle private score
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs*0.995)})
result.to_csv("Rossmann_submission_2.csv", index=False)

# model3 kaggle private score 
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs*w_test)})
result.to_csv("Rossmann_submission_3.csv", index=False)

In [ ]:
plt.figure(figsize=(10,6))
xgb.plot_importance(gbm)
plt.savefig("importance.png")

In [ ]:
# plot hold setb

res = res.sort_values(by=["Year", "Month", "Day"], ascending=[True, True, True])

col_1 = ['Sales_True','Prediction_True']
col_2 = ['Ratio_True']

L = np.random.randint( low=1,high = 1115, size = 3 ) 
print('Mean Ratio of predition and real sales data is {}: store all'.format(res["Ratio_True"].mean()))
for i in L:
    
    s1 = pd.DataFrame(res[res['Store']==i],columns = col_1)
    s2 = pd.DataFrame(res[res['Store']==i],columns = col_2)
    s1.plot(title = 'Comparation of predition and real sales data: store {}'.format(i),figsize=(12,4))
    s2.plot(title = 'Ratio of predition and real sales data: store {}'.format(i),figsize=(12,4))
    print('Mean Ratio of predition and real sales data is {}: store {}'.format(s2["Ratio_True"].mean(),i))